In [1]:
%matplotlib qt

In [8]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
import json
from sklearn import preprocessing

In [1]:
"""with open("MC3/MC3.json", "r") as f:
    data = json.load(f)

# Create a NetworkX graph
G = nx.MultiGraph()

# Add nodes to the graph
for i, node in enumerate(data['nodes']):
    node_id = node['id'][0]
    G.add_node(node_id, serial=i, type=node["type"][0])

    
# Add edges to the graph
for edge in data['links']:
    source = edge['source'][0]
    target = edge['target'][0]
    G.add_edge(source,target)"""

'with open("MC3/MC3.json", "r") as f:\n    data = json.load(f)\n\n# Create a NetworkX graph\nG = nx.MultiGraph()\n\n# Add nodes to the graph\nfor i, node in enumerate(data[\'nodes\']):\n    node_id = node[\'id\'][0]\n    G.add_node(node_id, serial=i, type=node["type"][0])\n\n    \n# Add edges to the graph\nfor edge in data[\'links\']:\n    source = edge[\'source\'][0]\n    target = edge[\'target\'][0]\n    G.add_edge(source,target)'

#### Convertir a numeros

In [5]:
df = pd.DataFrame(data["links"])

In [6]:
df.source = df.source.apply(lambda x: x[0])
df.target = df.target.apply(lambda x: x[0])

df = df[["source", "target"]]

In [9]:
encoder = preprocessing.OrdinalEncoder()

In [10]:
edges = encoder.fit_transform(df.values)

In [15]:
G = nx.MultiGraph()

for edge in edges:
    source = edge[0]
    target = edge[1]
    G.add_edge(source,target)

In [16]:
G.number_of_nodes(), G.number_of_edges()

(21265, 24038)

In [ ]:
nx.draw(G)

In [ ]:
nx.write_gexf(G, "graph.gexf") ##para pasar al formato de giphi

#### Para ver que hay nodos duplicados

In [46]:
df = pd.DataFrame(data["nodes"])

In [52]:
df.id = df.id.apply(lambda x:x[0])dd

In [ ]:
df

In [54]:
df.pivot_table(index=["id"],aggfunc ='size')

id
1 AS Marine sanctuary           1
1 Eel Corporation Transport     1
1 Ltd. Corporation Transport    1
1 Ltd. Liability Co             1
1 Ltd. Liability Co Cargo       1
                               ..
zhāng yú Marine conservation    1
zūn yú GmbH & Co. KG Creek      1
zūn yú Ltd. Liability Co        1
zūn yú N.V. Shipping            1
zūn yú S.A. de C.V.             1
Length: 22929, dtype: int64